In [127]:
##Import libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta

# import all libraries and dependencies for data visualization
pd.options.display.float_format='{:.4f}'.format
plt.rcParams['figure.figsize'] = [8,8]
pd.set_option('display.max_columns', 350)
pd.set_option('display.max_colwidth', -1) 
pd.set_option("display.max_rows", 50)
sns.set(style='darkgrid')
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker


# import all libraries and dependencies for machine learning
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.base import TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler

from sklearn.compose import make_column_transformer


import statsmodels.api as sm
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

from scipy import stats
from scipy.stats import norm, kurtosis, skew

# Import specific libraries
import statsmodels.api as sm
from statsmodels.stats import diagnostic as diag
from statsmodels.stats.outliers_influence import variance_inflation_factor


# Stats
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

from scipy import stats
from scipy.stats import skew, norm, probplot, boxcox
from scipy.special import boxcox1p
from patsy import dmatrices

pd.set_option('expand_frame_repr', False)




In [128]:
%store -r data_b
%store -r y_train
all_data = data_b.copy()


In [129]:
new_features = ["TotalPoints", "TotalExtraPoints", "GarageArea_x_Car", "TotalBsmtSF_x_Bsm", "TotalBsmtSF", 'ConstructArea',
               "TotalPorch", "GarageNewest", "LotAreaMultSlope", "BsmtSFPoints", 'BsmtSFMultPoints', 'BsmtSFMultPoints']

cat_ordinal = ["LotShape", 'LandSlope', 'ExterQual', 'ExterCond', 
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'HeatingQC', 
       'KitchenQual', 'Functional', 'FireplaceQu',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC',
       'Fence', "Street", 'CentralAir']



In [130]:
train = all_data.iloc[:len(y_train),:]
test = all_data.iloc[len(y_train):,:]



In [131]:
#Revisamos los features originales y filtramos aquellos con un skew > 0.5 en train:

col_skew = list(set(all_data.select_dtypes(exclude = "object").columns) - set(cat_ordinal) - set(new_features)) ##Dejo solo los numericos originales
sk = pd.DataFrame(columns = ["Feature", "Skew"])


#Filtro
for c in col_skew:
    sk = sk.append({"Feature":c, "Skew": train[c].skew()}, ignore_index = True)
sk[np.abs(sk["Skew"]) > 0.5].sort_values(by = "Skew", ascending = False)

Feature    Skew
21  PoolArea     15.6508
10  3SsnPorch    7.8993 
3   LowQualFinSF 7.5486 
6   MiscVal      5.0015 
2   KitchenAbvGr 4.0704 
4   BsmtHalfBath 3.9758 
28  ScreenPorch  3.3082 
18  MasVnrArea   0.9411 
32  2ndFlrSF     0.8804 
0   WoodDeckSF   0.7606 
31  OpenPorchSF  0.6137 
15  YearRemodAdd -0.5020

In [132]:
skewed_features = all_data[col_skew].apply(lambda x : skew (x.dropna())).sort_values(ascending=False)

#compute skewness
skewness = pd.DataFrame({'Skew' :skewed_features})   

# Get only higest skewed features
skewness = skewness[abs(skewness) > 0.5]
skewness = skewness.dropna()
print ("There are {} higest skewed numerical features to box cox transform".format(skewness.shape[0]))

l_opt = {}

for feat in skewness.index:
    all_data[feat], l_opt[feat] = boxcox((all_data[feat]+1))

skewed_features2 = all_data[skewness.index].apply(lambda x : skew (x.dropna())).sort_values(ascending=False)

#compute skewness
skewness2 = pd.DataFrame({'New Skew' :skewed_features2})   
display(pd.concat([skewness, skewness2], axis=1).sort_values(by=['Skew'], ascending=False))

There are 13 higest skewed numerical features to box cox transform


Skew  New Skew
PoolArea     22.1211 15.4625  
3SsnPorch    9.2211  8.8152   
LowQualFinSF 8.6154  8.4558   
MiscVal      5.0242  4.8178   
BsmtHalfBath 3.7496  3.7484   
KitchenAbvGr 3.6268  -0.9735  
ScreenPorch  3.3331  2.9135   
2ndFlrSF     1.2606  0.2928   
MasVnrArea   0.9576  0.4348   
WoodDeckSF   0.8428  0.1064   
OverallCond  0.6356  0.0090   
OpenPorchSF  0.6101  -0.1334  
Fireplaces   0.5527  0.0925

In [133]:
#Revisamos los features originales y filtramos aquellos con un skew > 0.5 en test:

col_skew = list(set(all_data.select_dtypes(exclude = "object").columns) - set(cat_ordinal) - set(new_features)) ##Dejo solo los numericos originales
sk = pd.DataFrame(columns = ["Feature", "Skew"])


#Filtro
for c in col_skew:
    sk = sk.append({"Feature":c, "Skew": test[c].skew()}, ignore_index = True)
sk[np.abs(sk["Skew"]) > 0.5].sort_values(by = "Skew", ascending = False)

Feature    Skew
21  PoolArea     15.4380
10  3SsnPorch    10.4110
3   LowQualFinSF 10.0216
6   MiscVal      4.8333 
4   BsmtHalfBath 3.5542 
28  ScreenPorch  2.7554 
7   TotBathrooms 0.6831 
5   OverallCond  -1.0322
2   KitchenAbvGr -2.1112

In [134]:
#Revisamos los features originales y filtramos aquellos con un skew > 0.5:

col_skew = list(set(all_data.select_dtypes(exclude = "object").columns) - set(cat_ordinal) - set(new_features)) ##Dejo solo los numericos originales
sk = pd.DataFrame(columns = ["Feature", "Skew"])


#Filtro
for c in col_skew:
    sk = sk.append({"Feature":c, "Skew": all_data.iloc[:len(y_train),:][c].skew()}, ignore_index = True)
sk[np.abs(sk["Skew"]) > 0.5].sort_values(by = "Skew", ascending = False)

Feature    Skew
21  PoolArea     15.5188
10  3SsnPorch    7.7858 
3   LowQualFinSF 7.4519 
6   MiscVal      4.8124 
4   BsmtHalfBath 3.9758 
28  ScreenPorch  3.0960 
15  YearRemodAdd -0.5020

In [135]:
h_skew = []
h_skew = [k for k in sk[np.abs(sk["Skew"]) > 0.5]["Feature"].unique()]
   


In [136]:
##Aplicaremos el BoxCox transformation a estos features para corregir su skew:

for k in h_skew:
    #print(k, boxcox_normmax(all_data[k] + 1))
    all_data.iloc[:len(y_train),:][k] = boxcox1p(all_data.iloc[:len(y_train),:][k], boxcox_normmax(all_data.iloc[:len(y_train),:][k] + 1))
    
# #Chequeamos nuevamente el skew:
col_skew = list(set(all_data.select_dtypes(exclude = "object").columns) - set(cat_ordinal) - set(new_features)) ##Dejo solo los numericos originales
s = pd.DataFrame(columns = ["Feature", "Skew"])
for c in col_skew:
    s = s.append({"Feature":c, "Skew": all_data.iloc[:len(y_train),:][c].skew()}, ignore_index = True)
s.sort_values(by = "Skew", ascending = False)

ValueError: array must not contain infs or NaNs

In [ ]:
##Se puede aplicar boxcox a toda la data (train +test)??
#Si no es así, hay que hacer el trabajo por separado